In [1]:
import json
import numpy as np
import pickle
import re
import string
import pandas as pd
import spacy
import re
import string
import functions as f
from glob import glob
from gensim import matutils, models
import scipy.sparse
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.feature_extraction import text 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
spacy.prefer_gpu()
import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)
import funcy as fp
from gensim import models
from gensim.corpora import Dictionary, MmCorpus
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
# sklearn implements t-SNE.
from sklearn.manifold import TSNE
from sklearn.preprocessing import scale
from sklearn.manifold.t_sne import _joint_probabilities, _kl_divergence
import seaborn as sns
import matplotlib.patheffects as PathEffects
import umap.umap_ as umap

/home/nick/anaconda3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.manifold.t_sne module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.manifold. Anything that cannot be imported from sklearn.manifold is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
with open('/home/nick/Documents/Reddit_NLP/pickles/ama.pickle', 'rb') as file:
    text = pickle.load(file)

In [3]:
stop_list = ['of', 'it', 'to', 'for', 'you', 'the', 'that', 'if', 'my', 'and', 'in', 'from','going', 'too', 'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa', 'aaaaaaaaaaaaaaaaaaaaaaaa', 'aaaaaaaaaaaa', 'aaaaaaaa', 'aaa', 'aaaaaaaaahhhhhhhhhhhh', 'aaaaaaaand', 'aaaaaaaauuuuuugggghhh', 'aaaaaaafternoon', 'aaaaaaand', 'aaaaaand', 'aaaaall', 'aaaaalllllll', 'aaaaand', 'is', 'are', 'what', 'they', 'we', 'about', 'me', 'have', 'im', 'this', 'yeah','will','can','get','be', 'have','be', 'but', 'he', 'she', 'her', 'was', 'with', 'they']

idf_df = f.vectorize(text, stop_words=stop_list)
idf_df

,00,000,10,100,1000,100k,101,10k,11,111,...,yuck,yum,yup,zadra,zakulon,zones,zoo,zookeeper,zoom,zoompleaños
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1880,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1881,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1882,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1883,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
lsa = TruncatedSVD(5)
doc_topic = lsa.fit_transform(idf_df)
lsa.explained_variance_ratio_

array([0.00554879, 0.0109853 , 0.01055856, 0.00839254, 0.00729809])

In [5]:
topic_word = pd.DataFrame(lsa.components_.round(3),
             columns = idf_df.columns)
topic_word

,00,000,10,100,1000,100k,101,10k,11,111,...,yuck,yum,yup,zadra,zakulon,zones,zoo,zookeeper,zoom,zoompleaños
0,0.001,0.001,0.023,0.003,0.001,0.002,0.001,0.001,0.003,0.0,...,0.001,0.001,0.001,0.001,0.003,0.002,0.001,0.002,0.002,0.003
1,-0.001,-0.001,-0.010,-0.001,-0.000,-0.001,0.000,-0.000,-0.001,-0.0,...,-0.000,0.000,-0.000,-0.000,0.005,0.001,-0.000,-0.001,0.003,0.002
2,-0.001,-0.000,-0.010,-0.001,-0.000,0.001,-0.000,-0.000,0.001,-0.0,...,-0.000,-0.000,-0.000,-0.000,0.000,-0.000,-0.000,-0.001,0.000,-0.000
3,-0.000,-0.000,-0.005,0.000,-0.000,-0.000,-0.000,0.000,-0.000,0.0,...,0.000,-0.000,-0.000,0.000,0.000,-0.000,0.000,-0.000,0.000,0.000
4,-0.001,-0.001,0.010,-0.004,-0.001,-0.003,-0.001,-0.001,-0.002,-0.0,...,-0.001,-0.002,-0.001,-0.001,0.001,0.001,-0.002,0.000,-0.001,-0.003


In [7]:
Vt = pd.DataFrame(doc_topic.round(5))
Vt

,0,1,2,3,4
0,0.12951,-0.05442,-0.02811,-0.02277,0.17527
1,0.14048,-0.02509,-0.02990,-0.00092,-0.10717
2,-0.00000,0.00000,-0.00000,0.00001,-0.00001
3,0.11983,-0.01638,-0.01621,-0.00016,-0.07912
4,0.19981,-0.06295,0.03267,0.00188,-0.08778
...,...,...,...,...,...
1880,0.15750,-0.02210,-0.02880,0.00227,-0.12107
1881,0.17893,-0.01814,-0.02271,0.01059,-0.15698
1882,0.08252,-0.00370,-0.00905,0.00240,-0.06907
1883,0.09966,-0.02029,-0.01090,0.00106,-0.07397


In [8]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [12]:
for n in range(9):
    lsa = TruncatedSVD(n)
    doc_topic = lsa.fit_transform(idf_df)
    lsa.explained_variance_ratio_
    topic_word = pd.DataFrame(lsa.components_.round(3), columns = idf_df.columns)
    print(display_topics(lsa, idf_df.columns, 5))
    print('_______________________________________________________________________________')

None
_______________________________________________________________________________

Topic  0
do, your, so, how, did
None
_______________________________________________________________________________

Topic  0
do, your, so, how, did

Topic  1
birthday, happy, day, thank, us
None
_______________________________________________________________________________

Topic  0
do, your, so, how, did

Topic  1
birthday, happy, day, us, thank

Topic  2
no, birthday, happy, not, 13
None
_______________________________________________________________________________

Topic  0
do, your, so, how, did

Topic  1
birthday, happy, day, thank, us

Topic  2
no, birthday, happy, not, 13

Topic  3
yes, most, times, please, some
None
_______________________________________________________________________________

Topic  0
do, your, so, how, did

Topic  1
birthday, happy, day, us, thank

Topic  2
no, birthday, happy, not, 13

Topic  3
yes, most, times, please, pleaseee

Topic  4
do, why, did, how, want
None


In [13]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

In [14]:
nmf = NMF(n_components=8, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(idf_df)

In [15]:
print("\nTopics in NMF model (generalized Kullback-Leibler divergence):")
tfidf_feature_names = idf_df.columns
print_top_words(nmf, tfidf_feature_names, 20)


Topics in NMF model (generalized Kullback-Leibler divergence):
Topic #0: like don on or time know now ve just say as who people so an one would any do at
Topic #1: birthday happy thank nice hope day friend cheers well us mate best great patient your bro cake wish oh stay
Topic #2: no 13 good not decent idea its hate problem virgin connection unfortunately really straight female expect sometimes am worries 16
Topic #3: yes some most actually times happened go very believe all sure them young didn almost room tried sadly hitler only
Topic #4: do why want think yourself see allegations did job years post these where human how false thumbs whatever marvellady24 five
Topic #5: how did deleted much feel where after your old exactly win before when long many money invest congrats make ok
Topic #6: not lol am because at stuff just got kids so half cool would never person when people made had right
Topic #7: your so thanks ama on re good sorry answer tell question true makes does guess favorit